In [1]:
import tensorflow as tf
import os
import cv2
from common import imgShow, autoResize, orcPreprocess, encodeText, pad_listints
from backbone.CNN import ConvBlock
from transformer.RNN import RecBlock
input_shape = [800, 100]
from data.data_loader import data_loader
import numpy as np
from tensorflow.keras.layers import Input
from tensorflow.keras import Model
from common import totalCER, one_hot_decoder


In [ ]:

#load data
images, labels, file_names = data_loader(prob = 10/103000)
label_lengths = [len(i) for i in labels]
labels = pad_listints([encodeText(label) for label in labels])


val_images, val_labels, _ = data_loader(prob = 10/103000)


In [ ]:
#config
ss = [[2, 2], [2, 2], [2, 1], [2, 1], [1, 1]]
ps = [[2, 2], [2, 2], [2, 1], [2, 1], [1, 1]]
CHAR_LIST = sorted("-ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ")
vocab_len = len(CHAR_LIST)
learning_rate=0.001
learning_rate_decay = 0.00001
print(vocab_len)

In [ ]:
#model
input_shape = (input_shape[1],input_shape[0])
c = ConvBlock(input_shape=input_shape,hiden=128,ps=ps,ss=ss)
r = RecBlock(vocab_len)
input = Input(shape = input_shape)
output = c.call(input)
output = r.call(output)
model = Model(inputs=input, outputs=output)
model.summary()


In [ ]:

# Lấy danh sách các biến cần cập nhật trong mô hình
trainable_vars = model.trainable_variables

# Sử dụng trình tối ưu hóa Adam
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Định nghĩa hàm mất mát và vòng lặp huấn luyện
def ctc_loss(labels, label_length, predictions, num_frames = 188):
    
    
    """length is vocab length + 1"""
    batch_size = labels.shape[0]
    logit_length = [num_frames] * batch_size
    predictions = tf.transpose(predictions, perm=[1, 0, 2])
    return tf.nn.ctc_loss(
      labels=labels,
      logits=predictions,
      label_length=label_length,
      logit_length=logit_length,
      blank_index=-1)

def train_step(images, labels, label_lengths):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = ctc_loss(labels, label_lengths, predictions)
    gradients = tape.gradient(loss, trainable_vars)
    optimizer.apply_gradients(zip(gradients, trainable_vars))
    return loss

In [ ]:

# tạo data set
# Huấn luyện mô hình với trình tối ưu hóa Adam
batch_size = 32
# Tạo một dataset từ dữ liệu đào tạo và chia thành các mini-batch
dataset = tf.data.Dataset.from_tensor_slices((images, labels, label_lengths))
dataset = dataset.shuffle(buffer_size=100000).batch(batch_size)

In [ ]:
# Huấn luyện mô hình với trình tối ưu hóa Adam
num_epochs = 20
batch_size = 32
# Tạo một dataset từ dữ liệu đào tạo và chia thành các mini-batch
dataset = tf.data.Dataset.from_tensor_slices((images, labels, label_lengths))
dataset = dataset.shuffle(buffer_size=10000).batch(batch_size)
for epoch in range(num_epochs):
    for images, labels, label_lengths in dataset:
        loss = train_step(images, labels, label_lengths)
    print(f'Epoch {epoch + 1}, Loss: {loss.numpy()}')

#train
minCER = 1e15
val_images = np.array(val_images)
num_epochs = 30
for epoch in range(num_epochs):
    for images, labels, label_lengths in dataset:
        loss = train_step(images, labels, label_lengths)
    print(f'Epoch {epoch + 1} , Loss: {loss.numpy()}')    
    learning_rate*=(1 - learning_rate_decay)
    optimizer.learning_rate.assign(learning_rate)
    if(epoch % 2 == 0):
        val_predictions = model.predict(val_images)
        totalCER = totalCER(val_labels, one_hot_decoder(val_predictions))
        if totalCER < minCER:
            minCER = totalCER
            model.save("myModel.h5")
        print(f'total CER = {totalCER}')